In [ ]:
import os
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

import sys
sys.path.append('../code/')
import glob
from data.dataset import InMemoryTrainDataset

from matplotlib import pyplot as plt

In [ ]:
df_rle = pd.read_csv('../input/train.csv')

In [ ]:
train_img_names = df_rle.id.unique()

In [ ]:
from data.transforms import HE_preprocess
TRAIN_SIZE = 256
viz_ds = InMemoryTrainDataset(train_img_names[:5],
                              df_rle,
                              train_tile_size=TRAIN_SIZE,
                              reduce_factor=4,
                              train_transfo=HE_preprocess(augment=True, visualize=True, size=TRAIN_SIZE),
                              valid_transfo=HE_preprocess(augment=False, visualize=True, size=TRAIN_SIZE),
                              train_path="../input/train/",
                              iter_per_epoch=100,
                              sampling_mode='centered',
                              use_external=None, #None, #0.2,
                              on_spot_sampling=0.99)

In [ ]:
# viz_ds.iter_per_epoch = 1000
# viz_ds.on_spot_sampling = 0.2
# viz_ds.sampling_mode='centered' # 'visible'
# viz_ds.train_tile_size=512
# viz_ds.train_transfo = HE_preprocess(augment=True, visualize=True, size=512)
# viz_ds.valid_transfo = HE_preprocess(augment=False, visualize=True, size=512)
# viz_ds.train(True)

In [ ]:
from utils.plots import plot_contours

viz_dl = DataLoader(viz_ds, batch_size=64, shuffle=True)
imgs, masks = next(iter(viz_dl))

plt.figure(figsize=(16, 16))
for i, (img, mask) in enumerate(zip(imgs, masks)):
    img = img.permute(1, 2, 0).numpy()  # .astype(np.uint8)
    plt.subplot(8, 8, i + 1)
    
    plot_contours(img.copy(), mask.numpy(), w=2)
    plt.axis("off")
    plt.subplots_adjust(wspace=None, hspace=None)
print(img.shape)

In [ ]:
viz_ds.train(False)

In [ ]:
from utils.plots import plot_contours

viz_dl = DataLoader(viz_ds, batch_size=64, shuffle=True)
imgs, masks = next(iter(viz_dl))

plt.figure(figsize=(16, 16))
for i, (img, mask) in enumerate(zip(imgs, masks)):
    img = img.permute(1, 2, 0).numpy()  # .astype(np.uint8)
    plt.subplot(8, 8, i + 1)
    
    plot_contours(img.copy(), mask.numpy(), w=2)
    plt.axis("off")
    plt.subplots_adjust(wspace=None, hspace=None)
print(img.shape)